In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pyo import *
from tensorflow import keras
import time
from pyo_callbacks import ListenToLoss
import load_models
import numpy as np

In [3]:
from pyo_callbacks import WeightsDense

In [4]:
x_train, y_train, x_test, y_test = load_models.get_mnist_data()

In [12]:
#make data unbalanced

sample = x_train[0]
exclude = y_train[0]
x_train = np.array([x for x, y in zip(x_train, y_train) if not np.array_equal(y, exclude)])
y_train = np.array([y for y in y_train if not np.array_equal(y, exclude)])
print(x_train.shape)
print(y_train.shape)

x_train = np.concatenate((x_train, sample.reshape((1,28,28,1))))
print(x_train.shape)
y_train = np.concatenate((y_train, exclude.reshape((1,10))))
print(y_train.shape)

labels = np.argmax(y_train, axis=1)

unique, counts = np.unique(labels, return_counts=True)

dict(zip(unique, counts))

(5409, 28, 28, 1)
(5409, 10)
(5410, 28, 28, 1)
(5410, 10)


{0: 592, 1: 677, 2: 618, 3: 617, 4: 560, 5: 563, 6: 557, 7: 646, 8: 579, 9: 1}

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [5]:
model = load_models.get_mnist_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
batch_size = 128
epochs = 8

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [96]:
#weights structure:

#dense
#list with two elements for weights and bias
#weights: (inputsize x outputsize)
#bias: (output size, )

#conv
#weights: (convsize, convsize, input channels, output channels)
#bias: output channels

In [102]:
class WeightsDense(keras.callbacks.Callback):

    """Sonifies the change of weights of a dense layer with as many streams as output neurons.
    The weights are averaged for each neuron.
    Frequency shifts after each epoch depend on how much the weights have changed during the last epoch.
    Oscillations created with an LFO depend on the amount the weights deviate from 0.
    """

    def __init__(self, which_layer=-1):

        #define layer to be sonified
        self.which_layer = which_layer

    def on_train_begin(self, logs=None):

        #get output size of model and generate harmonies respectively
        streams = self.model.layers[-1].output_shape[self.which_layer]
        harms = gen_harmonies(base_freq=200, ratio=[1,10,12,15,18], n_streams = streams)

        #get initial weights of layer
        init_weights = self.model.layers[self.which_layer].get_weights()[0]
        self.init_weights_mean = np.mean(init_weights, axis=0)

        #adjust lfo depending on deviation of mean from 0
        self.deviation = np.absolute(self.init_weights_mean) * 200
        self.lfo = LFO(freq=self.deviation.tolist(), type=1, mul=1000, add=1200)
        self.osc = Sine(freq=harms, mul=0.5)
        self.bp = ButBP(self.osc, freq=self.lfo)

        #initialize frequency shifter with 0 shift
        self.shift = FreqShift(self.bp, shift=0)
        #mix channels for audio output
        self.output = self.shift.mix(2).out()



    def on_epoch_end(self, epoch, logs=None):

        #get weights and mean of weights
        weights = self.model.layers[-1].get_weights()
#         print(weights[0])
        self.weights_mean = np.mean(weights[0], axis=0)
#         print(self.weights_mean)

        #adjust lfo depending on deviation of mean to 0
        self.deviation = np.absolute(self.weights_mean) * 200
        self.lfo.setFreq(self.deviation.tolist())

        #shift frequencies depending on how much weights differ from last epoch
        change = self.weights_mean - self.init_weights_mean
        shift_by = change*10000
        self.shift.setShift(shift_by.tolist())

        #remember weights for next epoch
        self.init_weights_mean = self.weights_mean


    def on_train_batch_begin(self, batch, logs=None):

        delay_by = int(logs.get('size') * 0.2)
        #leave some time until shifting back to harmonic frequencies
        if batch == delay_by:
            self.shift.setShift(0)
        
        
        

In [7]:
s = Server().boot()
s.amp=0.2
s.start()

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, 
          verbose=0, callbacks=[WeightsDense()])

s.stop()

Pyo warning: Portmidi warning: no midi device found!
Portmidi closed.


In [6]:
#visualize with Tensorboard
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, 
          verbose=0, callbacks=[keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=2, write_images=True)])

In [131]:
print(model.evaluate(x_test, y_test))

10000/10000 [==============================] - 2s 214us/sample - loss: 0.1502 - acc: 0.9545
[0.15021226827949286, 0.9545]
